In [2]:
#Installing the necessary libraries
!pip install requests
!pip install beautifulsoup4
!pip install nltk



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 333.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.5 MB 3.8 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.5 MB 2.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.5 MB 1.5 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.5 MB 1.4 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 1.6 MB/s eta 0:00:01
   ------------ --


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Importing the library functionalities
import re
import requests
from bs4 import BeautifulSoup
import time
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import urllib.robotparser

In [20]:
# Ensure stopwords and other resources are available
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anoop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anoop\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anoop\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [21]:
# Function to fetch the HTML content of a page
def fetch_page(url):
   try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return response.text
   except requests.exceptions.HTTPError as http_err:
       print(f"HTTP error occurred: {http_err}")
   except Exception as e:
    print(f"Other error occurred: {e}")


In [22]:
# Function to extract all links from a webpage
def extract_links(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = set()  # Using a set to avoid duplicate URLs
    for anchor in soup.find_all('a', href=True):
        full_url = urljoin(base_url, anchor['href'])
        links.add(full_url)
    return links


In [23]:
# Function to tokenize the text content of a webpage
def tokenize(text):
    tokens = word_tokenize(text)  # Using nltk to tokenize words
    return tokens

In [24]:
# Function to normalize tokens: lowercasing, removing stopwords, stemming/lemmatization
def normalize(tokens, use_stemming=False):
    tokens = [token.lower() for token in tokens]  # Convert all tokens to lowercase

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Apply stemming or lemmatization
    if use_stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    else:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens


In [25]:
# Function to check if crawling a URL is allowed by robots.txt
robots_cache = {}

def can_crawl(url, user_agent="*"):
    base_url = url.split("/")[0] + "//" + url.split("/")[2]
    if base_url in robots_cache:
        rp = robots_cache[base_url]
    else:
        rp = urllib.robotparser.RobotFileParser()
        robots_txt_url = base_url + "/robots.txt"
        rp.set_url(robots_txt_url)
        rp.read()
        robots_cache[base_url] = rp
    return rp.can_fetch(user_agent, url)


In [26]:
# Main crawler function
def crawl(seed_url, max_pages=10):
    to_crawl = [seed_url]  # List of pages to visit
    crawled = set()  # Set of pages already visited

    while to_crawl and len(crawled) < max_pages:
        url = to_crawl.pop(0)  # Get the next page to crawl
        if url in crawled:
            continue  # Skip if already crawled

        # Check if the URL is allowed to be crawled by robots.txt
        if not can_crawl(url):
            print(f"Blocked by robots.txt: {url}")
            continue

        print(f"Crawling: {url}")
        html_content = fetch_page(url)

        if html_content:
            # Extract the text content from the page
            soup = BeautifulSoup(html_content, 'html.parser')
            page_text = soup.get_text()

            # Tokenize and normalize the text content
            tokens = tokenize(page_text)
            normalized_tokens = normalize(tokens, use_stemming=True)  # You can set use_stemming=False for lemmatization

            print(f"Normalized Tokens: {normalized_tokens[:25]}")  # Print first 10 tokens for brevity

            # Extract and add new links to the to_crawl list
            links = extract_links(html_content, url)
            to_crawl.extend(links - crawled)  # Add new links to crawl
            crawled.add(url)

        time.sleep(0.4)  # Add a delay to avoid overwhelming servers

    print(f"Crawled {len(crawled)} pages")

# Start crawling from a seed URL
seed_url = "https://jasper-johansen.github.io/julia_documentation/"
crawl(seed_url)

Crawling: https://jasper-johansen.github.io/julia_documentation/
Normalized Tokens: ['julia', 'languag', 'document', 'julia', 'languag', 'document', 'introduct', 'get', 'start', 'comment', 'variabl', 'constant', 'datatyp', 'oper', 'control', 'statement', 'refer', 'introduct', 'julia', 'high-level', ',', 'high-perform', 'program', 'languag', 'design']


NameError: name 'urljoin' is not defined